In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json,re
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
from preparando_datos import sum_into_column,split_and_sum,get_min_max,compute_average
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline

from sklearn.base import BaseEstimator, TransformerMixin
from joblib import Parallel, delayed




df_train=pd.read_csv('dataset/origen.csv')
status_types = df_train['status'].unique().tolist()
status_types.remove(np.nan)
status_types.append('Missing')
status_types.append('Canceled')
""" 
for i, status in enumerate(status_types):
    status_types[i] = f"status_{status}" """
#take a 10% sample out of train
df_test =  df_train.sample(frac=0.1, random_state=43)
#drop the sample from train
df_train.drop(df_test.index, inplace=True)

X_test = df_test.drop(["averageRating",'Unnamed: 0'], axis=1)
X = df_train.drop(["averageRating",'Unnamed: 0'], axis=1)
y = df_train["averageRating"]
y_test = df_test['averageRating']

del df_test,df_train



In [2]:
X['runtimeMinutes'].replace(0, np.nan, inplace=True)
X['budget'].replace(0, np.nan, inplace=True)
X['revenue'].replace(0, np.nan, inplace=True)
X['titleType'].replace(0, np.nan, inplace=True)

print(X.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 879787 entries, 0 to 977540
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   numVotes              879787 non-null  int64  
 1   titleType             879787 non-null  object 
 2   isAdult               879787 non-null  float64
 3   startYear             879787 non-null  int64  
 4   endYear               879787 non-null  int64  
 5   runtimeMinutes        647113 non-null  float64
 6   genres_x              879786 non-null  object 
 7   directors             879787 non-null  object 
 8   writers               879787 non-null  object 
 9   seasonNumber          394643 non-null  float64
 10  episodeNumber         394643 non-null  float64
 11  ordering              333412 non-null  float64
 12  language              333412 non-null  object 
 13  attributes            333412 non-null  object 
 14  isOriginalTitle       333412 non-null  float64
 15  

In [3]:

categorical_columns_with_nan = [
    c for c in X.columns if X[c].dtype == 'O' and X[c].isnull().any()]

categorical_columns_without_nan = [
    c for c in X.columns if X[c].dtype == 'O' and X[c].notnull().all()]



numerical_columns_with_nan = [
    c for c in X.columns if X[c].dtype != 'O' and X[c].isnull().any()]
numerical_columns_without_nan = [
    c for c in X.columns if X[c].dtype != 'O' and X[c].notnull().all()]

In [4]:
from feature_engine.imputation import AddMissingIndicator, CategoricalImputer, MeanMedianImputer
from feature_engine.encoding import OneHotEncoder,RareLabelEncoder
from feature_engine.selection import  DropFeatures
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import FunctionTransformer
def interpolate_numericals(df, group, category, method='linear'):
    # Fill in missing values using interpolation
    df[category].replace(0, np.nan, inplace=True)
    df[category].interpolate(method=method, inplace=True)
    
    # Store the fitted values in a dictionary
    if group not in interpolate_numericals.fitted_values:
        interpolate_numericals.fitted_values[group] = {}
    interpolate_numericals.fitted_values[group][category] = df[category].copy()
    
    return df

# Initialize the fitted values dictionary
interpolate_numericals.fitted_values = {}
for to_remove in ['budget','revenue']:
    try:
        numerical_columns_with_nan.remove(to_remove)
    except ValueError:
        pass

def check(x):
    print(f"Number of missing values in titleType before categorical imputer: {x['titleType'].isna().sum()}")
    return x

pipeline_preprocessing = Pipeline([
    ('turnToNan', FunctionTransformer(lambda x: x.assign(runtimeMinutes=x['runtimeMinutes'].replace(0, np.nan),
                                                          budget=x['budget'].replace(0, np.nan),
                                                          revenue=x['revenue'].replace(0, np.nan)))),
    ('numericalMissingIndicator', AddMissingIndicator(variables=numerical_columns_with_nan)),
    ('categoricalMissingIndicator', AddMissingIndicator(variables=categorical_columns_with_nan)),
    ('runtimeTransform', FunctionTransformer(lambda x: interpolate_numericals(x, 'titleType', 'runtimeMinutes'))),
    ('budgetTransform', FunctionTransformer(lambda x: interpolate_numericals(x, 'titleType', 'budget'))),
    ('revenueTransform', FunctionTransformer(lambda x: interpolate_numericals(x, 'titleType', 'revenue'))),
    ('endYearsTransform', FunctionTransformer(lambda X: X.assign(endYear=X.apply(lambda x: x['startYear'] if x['endYear'] == 0 else x['endYear'], axis=1)), validate=False)),
    ('categoricalImputer', CategoricalImputer(variables=categorical_columns_with_nan)), 
    ('meanMedianImputer', MeanMedianImputer(imputation_method='median', variables=numerical_columns_with_nan)),
    #('tvCategories',RareLabelEncoder(tol=0.01,n_categories=6, variables=['titleType'],replace_with='Other_TV')),
])



In [5]:
class GetExpDict(BaseEstimator, TransformerMixin):
    #Voy a intentar explicar esto lo mejor que puedas
    def __init__(self, group:list, categories:list,targets:list=['']):
        #
        if type(group) == str:
            group = [group]
        if type(categories) == str:
            categories = [categories]
        if type(targets) == str:
            targets = [targets]
        
        self.group = group
        self.categories = categories
        self.targets = targets
    
    def fit(self, X, y=None):
        grupos = []
        for group in self.group:
            for category in self.categories:
                for target in self.targets:
                    categorias = X[category].unique().tolist()
                    categorias = [x.split(',') if type(x) == str else [] for x in categorias]
                    categorias = [item for sublist in categorias for item in sublist]
                    categorias = list(set(categorias))
                    grupo = X.groupby(group).apply(lambda x: [(g, v) for g, v in zip(x[category], x[target] if target!= 'exp' else [1]*len(x))]).to_dict()
                    if '0' in grupo:
                        del grupo['0']
                    grupos.append((group,grupo,categorias,target))
        self.diccionarios = Parallel(n_jobs=-1)(delayed(self._fit_group)(group, grupo, categorias,target) for group, grupo, categorias,target in grupos)
        return self
    
    def _fit_group(self, group, grupo, categorias,target):
        dict_limpio = {}
        for sub_grupo, shows in grupo.items():
            for dir in sub_grupo.split(','):
                dict_limpio.setdefault(dir, []).extend(shows)

        authors_xp = {}
        for sub_grupo, values in dict_limpio.items():
            cat_counts = {cat: [0, 0] for cat in categorias}
            for cat, count in ((v[0], v[1]) for v in values):

                if cat in cat_counts:
                    cat_counts[cat][0] += count
                    cat_counts[cat][1] += 1
            
            authors_xp[sub_grupo] = {cat: compute_average(cat_counts[cat]) for cat in categorias}

        return (group, authors_xp, categorias,target)
    
    
    def transform(self, X):
        #TODO: Adaptar el codigo para que diga el minimo de todas las categorias, no por categoria
        new_columns = {
            f'{group}_{target}_{category}': X[group].apply(sum_into_column, args=(diccionario, category,))
            for group, diccionario, categorias, target in self.diccionarios
            for category in categorias
        }
        new_columns.update({
            f'{group}_{target}_min': X[group].apply(get_min_max, args=(diccionario, 0, categorias,))
            for group, diccionario, categorias, target in self.diccionarios
        })
        new_columns.update({
            f'{group}_{target}_max': X[group].apply(get_min_max, args=(diccionario, 1, categoriass,))
            for group, diccionario, categoriass, target in self.diccionarios
        })
        X = pd.concat([X, pd.DataFrame(new_columns)], axis=1)
        return X




""" pipeline_exp = Pipeline([

])
 """

In [6]:
def okay(x):
    print("Okay")
    return x
def binarize_genres(X):
    mlb = MultiLabelBinarizer()
    binarized_genres = mlb.fit_transform(X['genres_x'].str.split(','))
    df_binarized_genres = pd.DataFrame(binarized_genres, columns=mlb.classes_)
    
    
    for column in status_types:
        if column not in df_binarized_genres.columns:
            df_binarized_genres[column] = 0
    df_binarized_genres = pd.concat([X, df_binarized_genres], axis=1)
    return df_binarized_genres

#TODO: Comparar a ver si es mejor ni usar order, season y episodeNumber

pipeline = Pipeline([('preprocessing',pipeline_preprocessing),

    ('set_exp_dict_genres', GetExpDict(group=['directors',"writers"], categories=['genres_x','titleType'],targets=['numVotes','exp','runtimeMinutes'])),

    ('binarizeGenres', FunctionTransformer(binarize_genres)),
    ('removeEmptyRows', FunctionTransformer(lambda x: x.dropna(subset=['directors']))),
    
    ('oneHotEncoder', OneHotEncoder(variables=['titleType',"status"])),

    ('removeCategorical',FunctionTransformer(lambda x: x.drop(x.select_dtypes(include=['object']).columns, axis=1))), 

    ])

In [7]:
pipeline.fit(X,y)

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('turnToNan',
                                  FunctionTransformer(func=<function <lambda> at 0x00000151EEFCBE20>)),
                                 ('numericalMissingIndicator',
                                  AddMissingIndicator(variables=['runtimeMinutes',
                                                                 'seasonNumber',
                                                                 'episodeNumber',
                                                                 'ordering',
                                                                 'isOriginalTitle',
                                                                 'popularity',
                                                                 'runtime'])),
                                 ('categoricalMissingIndicator',
                                  AddMissingIndicator(variable...
                            targets=['numVotes', 'exp', 'runtimeMinutes'])),
                ('binarizeGenres',
                 FunctionTransformer(func=<function binarize_genres at 0x00000151AC30C180>)),
                ('removeEmptyRows',
                 FunctionTransformer(func=<function <lambda> at 0x00000151EEFEED40>)),
                ('oneHotEncoder',
                 OneHotEncoder(variables=['titleType', 'status'])),
                ('removeCategorical',
                 FunctionTransformer(func=<function <lambda> at 0x00000151EEFEECA0>))])

In [11]:
jjjj = pipeline.transform(X)

In [8]:
X_pred = pd.read_csv('dataset/testear.csv').drop(['Unnamed: 0'],axis=1)

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
import numpy as np
from xgboost import XGBRegressor
import cupy as cp
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split



In [10]:
regr = XGBRegressor(n_estimators=1100,eta=0.025,
                        max_depth=19,gamma = 0.20,
                        colsample_bytree = 0.7,colsample_bylevel=0.7,colsample_bynode=0.8,
                        tree_method = 'hist',
                        max_cached_hist_node=262144) # type: ignore
#regr = XGBRegressor(n_estimators=1200, eta=0.35, max_depth=7, multi_strategy="multi_output_tree", min_child_weight=1, subsample=1, colsample_bytree=1, gamma=0, alpha=0)
print("Training a XGBRegressor")
regr.fit(pipeline.transform(X), y)
print("Finished training the XGBRegressor")



Training a XGBRegressor


XGBoostError: [01:28:07] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\common\io.h:232: bad_malloc: Failed to allocate 29969927264 bytes.

In [ ]:

X_test = pipeline.transform(X_test)

score = regr.score(X_test, y_test)

print(f"R^2 score on testing data: {score:.4f}") 
# RMSE
from sklearn.metrics import mean_squared_error

y_pred = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))
#print(set(X_pred.columns))

In [ ]:
X_pred = pipeline.transform(X_pred)
Yest = regr.predict(X_pred).clip(1, 10)
salida = pd.DataFrame(data={"averageRating": Yest})
salida.index = X_pred.index
salida.to_csv("predicciones/pred39.csv", sep=',',index=True,  index_label='Id')